# 99. スキップ接続の深層理解

## 学習目標

1. **スキップ接続**の種類と比較
2. **ResNet**のスキップ接続
3. **U-Net**のスキップ接続との違い
4. **勾配流の改善**

## 目次

1. [スキップ接続の種類](#section1)
2. [ResNet vs U-Net](#section2)
3. [勾配の流れ](#section3)
4. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)

<a id="section1"></a>
## 1. スキップ接続の種類

### 主な2種類

1. **加算（Addition）**: ResNet型 - $y = F(x) + x$
2. **連結（Concatenation）**: U-Net型 - $y = [F(x), x]$

In [ ]:
def compare_skip_types():
    """スキップ接続の種類を比較"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # ResNet型（加算）
    ax = axes[0]
    
    # ブロック
    rect1 = Rectangle((0.1, 0.6), 0.2, 0.15, facecolor='lightblue', edgecolor='blue', linewidth=2)
    rect2 = Rectangle((0.1, 0.3), 0.2, 0.15, facecolor='lightblue', edgecolor='blue', linewidth=2)
    ax.add_patch(rect1)
    ax.add_patch(rect2)
    
    ax.text(0.2, 0.675, 'Conv', ha='center', va='center', fontsize=11)
    ax.text(0.2, 0.375, 'Conv', ha='center', va='center', fontsize=11)
    
    # 矢印
    ax.annotate('', xy=(0.2, 0.6), xytext=(0.2, 0.45),
               arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    
    # スキップ接続
    ax.annotate('', xy=(0.5, 0.375), xytext=(0.5, 0.675),
               arrowprops=dict(arrowstyle='->', color='red', lw=2,
                              connectionstyle='arc3,rad=0.3'))
    
    # 加算記号
    circle = plt.Circle((0.35, 0.375), 0.03, facecolor='white', edgecolor='red', linewidth=2)
    ax.add_patch(circle)
    ax.text(0.35, 0.375, '+', ha='center', va='center', fontsize=14, color='red')
    
    ax.text(0.3, 0.9, 'ResNet型: 加算', ha='center', fontsize=14, fontweight='bold')
    ax.text(0.3, 0.1, 'y = F(x) + x\n入力と出力の次元が同じ', ha='center', fontsize=11)
    
    ax.set_xlim(0, 0.6)
    ax.set_ylim(0, 1)
    ax.axis('off')
    
    # U-Net型（連結）
    ax = axes[1]
    
    # エンコーダ
    rect1 = Rectangle((0.05, 0.6), 0.15, 0.2, facecolor='lightblue', edgecolor='blue', linewidth=2)
    ax.add_patch(rect1)
    ax.text(0.125, 0.7, 'Enc', ha='center', va='center', fontsize=11)
    
    # デコーダ
    rect2 = Rectangle((0.35, 0.6), 0.15, 0.2, facecolor='lightgreen', edgecolor='green', linewidth=2)
    ax.add_patch(rect2)
    ax.text(0.425, 0.7, 'Dec', ha='center', va='center', fontsize=11)
    
    # 連結後
    rect3 = Rectangle((0.35, 0.25), 0.25, 0.2, facecolor='lightyellow', edgecolor='orange', linewidth=2)
    ax.add_patch(rect3)
    ax.text(0.475, 0.35, 'Concat', ha='center', va='center', fontsize=11)
    
    # スキップ接続
    ax.annotate('', xy=(0.35, 0.35), xytext=(0.2, 0.6),
               arrowprops=dict(arrowstyle='->', color='red', lw=2))
    ax.annotate('', xy=(0.35, 0.35), xytext=(0.425, 0.6),
               arrowprops=dict(arrowstyle='->', color='green', lw=2))
    
    ax.text(0.3, 0.9, 'U-Net型: 連結', ha='center', fontsize=14, fontweight='bold')
    ax.text(0.3, 0.1, 'y = [Enc(x), Dec(x)]\nチャンネル数が増加', ha='center', fontsize=11)
    
    ax.set_xlim(0, 0.6)
    ax.set_ylim(0, 1)
    ax.axis('off')
    
    plt.suptitle('スキップ接続の2つの種類', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

compare_skip_types()

<a id="section2"></a>
## 2. ResNet vs U-Net

In [ ]:
def compare_resnet_unet():
    """ResNetとU-Netの比較"""
    print("="*70)
    print("ResNet vs U-Net のスキップ接続比較")
    print("="*70)
    
    comparison = [
        ('', 'ResNet', 'U-Net'),
        ('接続タイプ', '加算', '連結'),
        ('次元変化', '同じ', '増加'),
        ('目的', '勾配の流れ改善', '詳細情報の保持'),
        ('接続範囲', '同一層内（残差）', 'エンコーダ→デコーダ'),
        ('計算量', '少ない', '多い（チャンネル増）'),
        ('主な用途', '画像分類', 'セグメンテーション'),
    ]
    
    for row in comparison:
        print(f"{row[0]:<15} {row[1]:<20} {row[2]:<20}")

compare_resnet_unet()

<a id="section3"></a>
## 3. 勾配の流れ

In [ ]:
def explain_gradient_flow():
    """スキップ接続による勾配の流れの改善"""
    print("="*60)
    print("スキップ接続と勾配の流れ")
    print("="*60)
    
    print("""
【スキップ接続なし】
  ∂L/∂x = ∂L/∂y × ∂y/∂x
  
  問題：深い層で勾配が消失または爆発

【ResNetのスキップ接続あり】
  y = F(x) + x
  
  ∂L/∂x = ∂L/∂y × (∂F/∂x + 1)
         = ∂L/∂y × ∂F/∂x + ∂L/∂y
  
  → +1項により、勾配が直接伝播
  → 勾配消失を防ぐ「高速道路」

【効果】
  ・非常に深いネットワーク（100層以上）が学習可能に
  ・恒等写像が容易に学習できる
  ・精度向上と学習の安定化
    """)

explain_gradient_flow()

<a id="summary"></a>
## 4. まとめ

### スキップ接続のポイント

1. **加算型（ResNet）**: 勾配の流れを改善、深いネットワークを可能に
2. **連結型（U-Net）**: 詳細情報を保持、セグメンテーションに有効
3. **共通の利点**: 情報の直接伝達、勾配消失の緩和